In [13]:
!pip install autogluon

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error

 ### 데이터 불러오기

In [4]:
summer = pd.read_csv("/content/drive/MyDrive/날씨 빅데이터 공모전/data/summer_train.csv")
summer_test = pd.read_csv("/content/drive/MyDrive/날씨 빅데이터 공모전/data/summer_test.csv")

### mmddhh 칼럼 월 일 시간으로 나누기

In [5]:
summer["hour"] = summer["mmddhh"] % 100
summer["day"] = (np.trunc((summer["mmddhh"] % 10000) / 100))
summer["month"] = np.trunc(summer["mmddhh"] / 10000)

summer_test["hour"] = summer_test["mmddhh"] % 100
summer_test["day"] = (np.trunc((summer_test["mmddhh"] % 10000) / 100))
summer_test["month"] = np.trunc(summer_test["mmddhh"] / 10000)

### 지면온도 예측에 대한 RN(누적강수량)의 일부 값을 변경

In [6]:
summer = summer.replace({"1시간 누적 강수량(mm)" : [1.8, 3.3, 5.3, 19.5, 30.3, 623.5]}, -99.9)

summer_test = summer_test.replace({"1시간 누적 강수량(mm)" : [1.8, 3.3, 5.3, 19.5, 30.3, 623.5]}, -99.9)

### 1시간 누적 일사량과 1시간 누적 일조량 칼럼 수정

In [7]:
#여름

summer_5 = [0, 1, 2, 3, 4, 5, 21, 22, 23]
summer_6 = [0, 1, 2, 3, 4, 5, 21, 22, 23]
summer_7 = [0, 1, 2, 3, 4, 5, 21, 22, 23]

for i in summer_5 :
    summer.loc[(summer["1시간 누적 일사량(MJ)"] == -99.9) & (summer["hour"] == i) & (summer["월"] == 5), "1시간 누적 일사량(MJ)"] = 0
    summer.loc[(summer["1시간 누적 일조량(초)"] == -99.9) & (summer["hour"] == i) & (summer["월"] == 5), "1시간 누적 일조량(초)"] = 0

for i in summer_6 :
    summer.loc[(summer["1시간 누적 일사량(MJ)"] == -99.9) & (summer["hour"] == i) & (summer["월"] == 6), "1시간 누적 일사량(MJ)"] = 0
    summer.loc[(summer["1시간 누적 일조량(초)"] == -99.9) & (summer["hour"] == i) & (summer["월"] == 6), "1시간 누적 일조량(초)"] = 0

for i in summer_7 :
    summer.loc[(summer["1시간 누적 일사량(MJ)"] == -99.9) & (summer["hour"] == i) & (summer["월"] == 7), "1시간 누적 일사량(MJ)"] = 0
    summer.loc[(summer["1시간 누적 일조량(초)"] == -99.9) & (summer["hour"] == i) & (summer["월"] == 7), "1시간 누적 일조량(초)"] = 0

for i in summer_5 :
    summer_test.loc[(summer_test["1시간 누적 일사량(MJ)"] == -99.9) & (summer_test["hour"] == i) & (summer_test["월"] == 5), "1시간 누적 일사량(MJ)"] = 0
    summer_test.loc[(summer_test["1시간 누적 일조량(초)"] == -99.9) & (summer_test["hour"] == i) & (summer_test["월"] == 5), "1시간 누적 일조량(초)"] = 0

for i in summer_6 :
    summer_test.loc[(summer_test["1시간 누적 일사량(MJ)"] == -99.9) & (summer_test["hour"] == i) & (summer_test["월"] == 6), "1시간 누적 일사량(MJ)"] = 0
    summer_test.loc[(summer_test["1시간 누적 일조량(초)"] == -99.9) & (summer_test["hour"] == i) & (summer_test["월"] == 6), "1시간 누적 일조량(초)"] = 0

for i in summer_7 :
    summer_test.loc[(summer_test["1시간 누적 일사량(MJ)"] == -99.9) & (summer_test["hour"] == i) & (summer_test["월"] == 7), "1시간 누적 일사량(MJ)"] = 0
    summer_test.loc[(summer_test["1시간 누적 일조량(초)"] == -99.9) & (summer_test["hour"] == i) & (summer_test["월"] == 7), "1시간 누적 일조량(초)"] = 0


summer.loc[(summer["1시간 누적 일사량(MJ)"] == -99.9) & (summer["1시간 누적 일조량(초)"] == 0), "1시간 누적 일사량(MJ)"] = 0
summer.loc[(summer["1시간 누적 일사량(MJ)"] == 0) & (summer["1시간 누적 일조량(초)"] == -99.9), "1시간 누적 일조량(초)"] = 0

summer_test.loc[(summer_test["1시간 누적 일사량(MJ)"] == -99.9) & (summer_test["1시간 누적 일조량(초)"] == 0), "1시간 누적 일사량(MJ)"] = 0
summer_test.loc[(summer_test["1시간 누적 일사량(MJ)"] == 0) & (summer_test["1시간 누적 일조량(초)"] == -99.9), "1시간 누적 일조량(초)"] = 0

### 결측치 처리

In [8]:
summer = summer.replace([-99, -99.9, -999], np.nan)

summer_test = summer_test.replace([-99, -99.9, -999], np.nan)

### 사용하지 않는 칼럼 제거

In [9]:
summer = summer.drop(["mmddhh", "적설 깊이(cm)", "월"], axis = 1)

summer_test = summer_test.drop(["mmddhh", "적설 깊이(cm)", "월"], axis = 1)

### 1시간 누적 강수유무와 1시간 누적 강수량 칼럼 수정

In [10]:
summer.loc[(summer["1시간 누적 강수유무(분)"]== 0) & (summer["1시간 누적 강수량(mm)"].isnull()), "1시간 누적 강수량(mm)"] = 0
summer.loc[summer["1시간 누적 강수량(mm)"].isnull(), "1시간 누적 강수량(mm)"] = 0
summer = summer.drop("1시간 누적 강수유무(분)", axis = 1)

summer_test.loc[(summer_test["1시간 누적 강수유무(분)"]== 0) & (summer_test["1시간 누적 강수량(mm)"].isnull()), "1시간 누적 강수량(mm)"] = 0
summer_test.loc[summer_test["1시간 누적 강수량(mm)"].isnull(), "1시간 누적 강수량(mm)"] = 0
summer_test = summer_test.drop("1시간 누적 강수유무(분)", axis = 1)

### 인덱스 초기화

In [11]:
summer = summer.reset_index(drop=True)

### 라벨 인코딩 수행

In [12]:
cha = ["지점", "연도", "현천"]

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in cha:
    summer[i] = le.fit_transform(summer[i])

    for label in np.unique(summer_test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)

    summer_test[i] = le.transform(summer_test[i])

<ipython-input-12-64fd08e218fe>:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if label not in le.classes_:



## 파생변수 추가

### 1. 기온 변화량

In [14]:
# 파생변수 추가
summer = summer.reset_index(drop = True)
summer_test = summer_test.reset_index(drop=True)

summer["기온변화량"] = -(summer["1시간 평균 기온(C)"] - summer["1시간 평균 기온(C)"][1:].reset_index(drop = True))
summer["기온변화량"] = summer["기온변화량"].shift(1)

summer_test["기온변화량"] = -(summer_test["1시간 평균 기온(C)"] - summer_test["1시간 평균 기온(C)"][1:].reset_index(drop = True))
summer_test["기온변화량"] = summer_test["기온변화량"].shift(1)



## 파생변수 추가

### 1. 기온 변화량

In [15]:
# 파생변수 추가
summer = summer.reset_index(drop = True)
summer_test = summer_test.reset_index(drop=True)

summer["기온변화량"] = -(summer["1시간 평균 기온(C)"] - summer["1시간 평균 기온(C)"][1:].reset_index(drop = True))
summer["기온변화량"] = summer["기온변화량"].shift(1)

summer_test["기온변화량"] = -(summer_test["1시간 평균 기온(C)"] - summer_test["1시간 평균 기온(C)"][1:].reset_index(drop = True))
summer_test["기온변화량"] = summer_test["기온변화량"].shift(1)


### 2. 상대습도 변화량

In [19]:
summer["상대습도변화량"] = -(summer["1시간 평균 상대습도(%)"] - summer["1시간 평균 상대습도(%)"][1:].reset_index(drop = True))
summer["상대습도변화량"] = summer["상대습도변화량"].shift(1)

summer_test["상대습도변화량"] = -(summer_test["1시간 평균 상대습도(%)"] - summer_test["1시간 평균 상대습도(%)"][1:].reset_index(drop = True))
summer_test["상대습도변화량"] = summer_test["상대습도변화량"].shift(1)


### 3. 평균 기온 * 평균 상대습도

In [20]:
summer["평균기온*평균상대습도"] = summer["1시간 평균 기온(C)"] * summer["1시간 평균 상대습도(%)"]

summer_test["평균기온*평균상대습도"] = summer_test["1시간 평균 기온(C)"] * summer_test["1시간 평균 상대습도(%)"]


### 4. 1시간 누적 일사량 * 1시간 누적 일조량

In [22]:
summer["1시간 누적 일사량 * 1시간 누적 일조량"] = summer["1시간 누적 일사량(MJ)"] * summer["1시간 누적 일조량(초)"]

summer_test["1시간 누적 일사량 * 1시간 누적 일조량"] = summer_test["1시간 누적 일사량(MJ)"] * summer_test["1시간 누적 일조량(초)"]

In [23]:
summer

,지점,연도,1시간 평균 기온(C),1시간 평균 이슬점온도(C),1시간 평균 상대습도(%),1시간 평균풍속(m/s),1시간 누적 강수량(mm),현천,1시간 평균 지면온도(C),1시간 누적 일사량(MJ),1시간 누적 일조량(초),hour,day,month,기온변화량,평균기온*평균상대습도,상대습도변화량,1시간 누적 일사량 * 1시간 누적 일조량
0,0,0,17.6,12.0,69.8,0.7,0.0,3,17.1,0.00,0.0,0,1.0,5.0,NaN,1228.48,NaN,0.000
1,0,0,15.9,12.3,79.2,0.7,0.0,3,16.4,0.00,0.0,1,1.0,5.0,-1.7,1259.28,9.4,0.000
2,0,0,15.0,12.3,84.0,0.8,0.0,3,15.9,0.00,0.0,2,1.0,5.0,-0.9,1260.00,4.8,0.000
3,0,0,15.1,8.2,65.4,1.7,0.0,3,15.4,0.00,0.0,3,1.0,5.0,0.1,987.54,-18.6,0.000
4,0,0,16.1,1.6,37.5,2.7,0.0,0,14.6,0.00,0.0,4,1.0,5.0,1.0,603.75,-27.9,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110395,9,4,29.4,25.9,81.6,4.6,0.0,0,30.6,0.11,0.9,19,31.0,7.0,-0.9,2399.04,4.9,0.099
110396,9,4,28.6,26.2,87.3,3.9,0.0,0,29.1,0.00,0.0,20,31.0,7.0,-0.8,2496.78,5.7,0.000
110397,9,4,28.2,26.1,88.6,3.5,0.0,0,28.3,0.00,0.0,21,31.0,7.0,-0.4,2498.52,1.3,0.000
110398,9,4,28.0,26.2,90.1,4.5,0.0,0,27.9,0.00,0.0,22,31.0,7.0,-0.2,2522.80,1.5,0.000


### 2개 이상 연속되는 결측치 찾기 및 제거

In [24]:
lists = ["1시간 평균 기온(C)", "1시간 평균 이슬점온도(C)", "1시간 평균 상대습도(%)", "1시간 평균풍속(m/s)" , "1시간 평균 지면온도(C)", "1시간 누적 일사량(MJ)", "1시간 누적 일조량(초)"
,"기온변화량" ,"상대습도변화량", "평균기온*평균상대습도", "1시간 누적 일사량 * 1시간 누적 일조량"]
for i in lists:
    ans = summer[summer[i].isnull()].index
    cons = []
    for j in range(len(ans)-1) :
        if ans[j+1] - ans[j] == 1 : # i+1번째 숫자와 i번째 숫자의 차이가 1이라면(연속이라면)
            cons.extend([ans[j],ans[j+1]]) #i+1번째 숫자와 i번째 숫자를 결과 출력할 리스트에 추가
    delete = list(set(cons)) #위 알고리즘으로 하면 만일 숫자가 23,24,25라면 23, 24, 24, 25로 저장되기 때문에 중복되는 숫자를 제거

    summer = summer.drop(summer.index[delete])
    #인덱스를 초기화
    summer = summer.reset_index(drop=True)

In [25]:
summer

,지점,연도,1시간 평균 기온(C),1시간 평균 이슬점온도(C),1시간 평균 상대습도(%),1시간 평균풍속(m/s),1시간 누적 강수량(mm),현천,1시간 평균 지면온도(C),1시간 누적 일사량(MJ),1시간 누적 일조량(초),hour,day,month,기온변화량,평균기온*평균상대습도,상대습도변화량,1시간 누적 일사량 * 1시간 누적 일조량
0,0,0,17.6,12.0,69.8,0.7,0.0,3,17.1,0.00,0.0,0,1.0,5.0,NaN,1228.48,NaN,0.000
1,0,0,15.9,12.3,79.2,0.7,0.0,3,16.4,0.00,0.0,1,1.0,5.0,-1.7,1259.28,9.4,0.000
2,0,0,15.0,12.3,84.0,0.8,0.0,3,15.9,0.00,0.0,2,1.0,5.0,-0.9,1260.00,4.8,0.000
3,0,0,15.1,8.2,65.4,1.7,0.0,3,15.4,0.00,0.0,3,1.0,5.0,0.1,987.54,-18.6,0.000
4,0,0,16.1,1.6,37.5,2.7,0.0,0,14.6,0.00,0.0,4,1.0,5.0,1.0,603.75,-27.9,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109712,9,4,29.4,25.9,81.6,4.6,0.0,0,30.6,0.11,0.9,19,31.0,7.0,-0.9,2399.04,4.9,0.099
109713,9,4,28.6,26.2,87.3,3.9,0.0,0,29.1,0.00,0.0,20,31.0,7.0,-0.8,2496.78,5.7,0.000
109714,9,4,28.2,26.1,88.6,3.5,0.0,0,28.3,0.00,0.0,21,31.0,7.0,-0.4,2498.52,1.3,0.000
109715,9,4,28.0,26.2,90.1,4.5,0.0,0,27.9,0.00,0.0,22,31.0,7.0,-0.2,2522.80,1.5,0.000


### 결측치 채우기 : 선형 보간

In [26]:
summer = summer.interpolate(method = 'linear')

summer_test = summer_test.interpolate(method = 'linear')

### autogluon 적용

In [27]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [28]:
# 0~2년도의 데이터를 train, 3년도의 데이터를 validation, 4년도의 데이터를 test로 하여 autogluon 적용

auto_summer_train = summer.loc[(summer["연도"] != 3) & (summer["연도"] != 4)]
auto_summer_val = summer.loc[summer["연도"] == 3]
auto_summer_test = summer.loc[summer["연도"] == 4]

In [29]:
predictor = TabularPredictor(label="1시간 평균 지면온도(C)", eval_metric = "mae").fit(auto_summer_train, tuning_data = auto_summer_val)

#predictor_1 = TabularPredictor(label="1시간 평균 지면온도(C)", eval_metric = "mae").fit(x_train, tuning_data = x_val)

No path specified. Models will be saved in: "AutogluonModels/ag-20230705_044022/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230705_044022/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Disk Space Avail:   83.44 GB / 115.66 GB (72.1%)
Train Data Rows:    65724
Train Data Columns: 17
Tuning Data Rows:    22018
Tuning Data Columns: 17
Label Column: 1시간 평균 지면온도(C)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (64.6, 3.6, 26.08643, 9.47775)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preproces

In [30]:
predictor.leaderboard(auto_summer_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-1.712086,-1.575896,0.698263,0.664217,5.656647,0.698263,0.664217,5.656647,1,True,3
1,WeightedEnsemble_L2,-1.726643,-1.520246,6.316745,4.024248,432.880869,0.008892,0.009183,0.704548,2,True,12
2,RandomForestMSE,-1.803604,-1.579245,2.261542,1.569138,193.314451,2.261542,1.569138,193.314451,1,True,5
3,ExtraTreesMSE,-1.806407,-1.580156,2.383124,1.211689,67.852421,2.383124,1.211689,67.852421,1,True,7
4,LightGBMLarge,-1.813304,-1.596155,0.337205,0.252323,3.527402,0.337205,0.252323,3.527402,1,True,11
5,XGBoost,-1.816790,-1.551755,0.120110,0.121635,3.909384,0.120110,0.121635,3.909384,1,True,9
6,CatBoost,-1.822737,-1.595830,0.078266,0.015239,10.729887,0.078266,0.015239,10.729887,1,True,6
7,LightGBM,-1.842035,-1.593720,0.267184,0.261949,2.538229,0.267184,0.261949,2.538229,1,True,4
8,NeuralNetFastAI,-2.168112,-1.731793,0.665711,0.348281,94.801396,0.665711,0.348281,94.801396,1,True,8
9,NeuralNetTorch,-2.173137,-1.914995,0.179103,0.100104,66.642021,0.179103,0.100104,66.642021,1,True,10


### summer_test에 적용

In [31]:
summer_pred = predictor.predict(summer_test)
#summer_pred = predictor_1.predict(summer_test)

In [32]:
summer_pred

0        9.130077
1        8.687352
2        8.525805
3        7.949724
4        7.704664
          ...    
6619    28.217800
6620    27.789455
6621    27.450224
6622    27.522961
6623    27.292017
Name: 1시간 평균 지면온도(C), Length: 6624, dtype: float32

In [33]:
import pandas as pd

# 파일명
file_name = '/content/drive/MyDrive/날씨 빅데이터 공모전/data/subminssionUser.xlsx'

writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
submission_summer = pd.read_excel(file_name, sheet_name= "SUMMER")

# Daraframe형식으로 엑셀 파일 읽기
submission_summer = pd.read_excel(file_name, sheet_name= "SUMMER")

#제출 파일에 빠진 날짜 찾아서 테스트에서도 빼버리기
summerindexdrop = [879,880,3084,3180,5292]
summer_pred = summer_pred.drop(summerindexdrop)
submission_summer["TS"] = summer_pred

submission_summer.to_excel(
    writer,
    sheet_name='SUMMER',
    index=False,
    # header = None
    )
writer.save()
writer.close()

<ipython-input-33-7885c0900bd9>:23: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


이상치라고 판단해서 데이터를 제거하고 돌리면 오히려 오차값이 상승한다
따라서 데이터를 제거하는 방법은 맞지 않아 보이고
결측치를 대체할 수 없어서 데이터를 제거한 부분까지도 제거하지 않고 최대한 데이터를 대체하는 형식으로 한다면 오차를 더 줄여볼수도?